In [3]:
import transformers 
from transformers import pipeline
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import torch

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [9]:
named_entity_recognition_pipeline = pipeline("ner", aggregation_strategy="simple", model="Jean-Baptiste/roberta-large-ner-english", device=device)

In [23]:
zero_shot_classifier_pipeline = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

In [34]:
text_input = "things to do Dallas."

# first get all named entities from the text input
entities = named_entity_recognition_pipeline(text_input)
# get all location entities
location_entities = [entity for entity in entities if entity['entity_group']=='LOC']
subject = location_entities[0]['word'] if len(location_entities)!=0 else subject # replace subject is there is a new location entity


# next, we need to figure out what the user is asking for
# major categories, text info, videos, images, tweets, 
major_categories = ["videos", "images", "tweets", "info"]
classes = zero_shot_classifier_pipeline(text_input, candidate_labels=major_categories)
major_category = classes['labels'][0]
major_category_score = classes['scores'][0]


# next, if we have a request for info, we need to break down what info they are asking for,
info_categories = []
if major_category == "info":
    print("getting more info")
    classes = zero_shot_classifier_pipeline(text_input, candidate_labels=info_categories)
    major_category = classes['labels'][0]
    major_category_score = classes['scores'][0]
    
subject, major_category, major_category_score

('info', 0.8277179002761841)